# How to generate input and output data for our model

## Inputs

- **Geometry**: a set of primitives (triangles) with
  + variable size (number of primitives can vary from inputs);
  + shape of `[n_primitives, 3, coordinates : 3]`;
  + primitives order is **not** important.
- **TX** and **RX**: transmitting and receiving nodes with
  + variables number of **TX** or **RX**;
  + each node has shape `[coordinates : 3`];
  
> NOTE: the problem should be separable for each pair of (TX, RX). I.e., addressing the problem of path finding as if there were only one TX and one RX is ok.

## Output

- **Paths**: a sequence of sequences of coordinates such that:
  + the number of paths (i.e., the size of outer sequence) is variable;
  + the size of a given path (i.e., the size of any inner sequence) is variable, but can be **bounded** by some factor, usually referred to as the `max_depth`.
  
> NOTE: it could be interesting to favor the paths that have the shortest length.
  
In practive, paths are stored in a continuous tensor in memory. As such, Sionna uses a padding value to indicate missing path: `0.0`. This is useful, e.g., if there exists 5 paths from `tx_0` to `rx_0`, but only 3 paths from `tx_0` to `rx_1`.

For shorter paths, i.e., paths that have less than `max_depth` interactions, one must use some post-processing to remove invalid vertices. An example of such post-processing can be found in the function [`sionna.rt.utils.paths_to_segments`](https://github.com/jeertmans/sionna/blob/machine-learning/sionna/rt/utils.py#L270).

In [ ]:
import sionna
import tensorflow as tf

# Import Sionna RT components
from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter

# For pretty printing
from textwrap import indent

In [ ]:
seed = 12

# Load integrated scene

scene_name = "simple"

if scene_name == "simple":
    scene = load_scene(sionna.rt.scene.simple_street_canyon)
elif scene_name == "etoile":
    scene = load_scene(sionna.rt.scene.etoile)
elif scene_name == "munich":
    scene = load_scene(sionna.rt.scene.munich)
else:
    raise ValueError(f"Unknown scene '{scene_name}'")

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1, 
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="cross")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="cross")

# Too large values can cause OOM errors (or lags) on
# larger scenes.
n_sources = 2
n_targets = 2

min_x, min_y, min_z = scene.mi_scene.bbox().min
max_x, max_y, max_z = scene.mi_scene.bbox().max

max_z += 10

for i, x, y, z in zip(
    range(n_sources),
    tf.random.uniform((n_sources,), minval=min_x, maxval=max_x, seed=seed),
    tf.random.uniform((n_sources,), minval=min_y, maxval=max_y, seed=seed),
    tf.random.uniform((n_sources,), minval=min_z, maxval=max_z, seed=seed),
):
    # Create transmitter
    tx = Transmitter(name=f"tx_{i}",
                     position=[x, y, z])

    # Add transmitter instance to scene
    scene.add(tx)
    
for i, x, y, z in zip(
    range(n_targets),
    tf.random.uniform((n_targets,), minval=min_x, maxval=max_x, seed=seed),
    tf.random.uniform((n_targets,), minval=min_y, maxval=max_y, seed=seed),
    tf.random.uniform((n_targets,), minval=min_z, maxval=max_z, seed=seed),
):
    # Create receiver
    rx = Receiver(name=f"rx_{i}",
                  position=[x, y, z])

    # Add receiver instance to scene
    scene.add(rx)

scene.preview()

# Current Inputs

In [ ]:
# First input: geometry primitives
primitives = scene._solver._primitives
print(primitives.shape)

# NOTE: Python's builtin dict should keep the insert ordering when iterating over transmitters
#       and receivers. So that the ith transmitters if tx_i.
#       This invariant is important since we use indexing in the paths to make a given path
#       correspond to as given (tx, rx) pair.

i = 0
j = 1

# Second input
tx = scene.transmitters[f"tx_{i}"].position
rx = scene.receivers[f"rx_{i}"].position

print(f"tx_{i} located at", tx.numpy())
print(f"rx_{i} located at", rx.numpy())

In [ ]:
# Compute propagation paths
paths = scene.compute_paths(max_depth=3,
                            #method="exhaustive",
                            method="stochastic", # For small scenes the method can be also set to "exhaustive"
                            num_samples=1e6,     # Number of rays shot into random directions, too few rays can lead to missing paths
                            seed=seed)           # By fixing the seed, reproducible results can be ensured

scene.preview(paths, show_devices=True, show_paths=True) # Use the mouse to focus on the visualized paths

# Current Output

In [ ]:
# [max_depth, num_targets, num_sources, max_num_paths, coordinates : 3]
# max_num_paths cannot be predicted, but is always <= num_primitives**max_depth + 1
vertices = paths.vertices

_, _, _, max_num_paths, _ = vertices.shape

print("Output's shape over all (tx,rx) pairs:", vertices.shape)

print(f"tx_{i} located at", tx.numpy())
print(f"rx_{i} located at", rx.numpy())

# IMPORTANT: paths must be post-processed, see comment above.
for k in range(max_num_paths):
    path = vertices[:, j, i, k, :]
    pretty_path = indent(repr(path), "\t")
    print(f"\nPath {k} from tx_{i} to rx_{j} is", pretty_path, sep="\n")